In [1]:
import numpy as np
from numpy import array, zeros, diag, ones, sin, cos, tan, linspace, dot, pi
from numpy.random import uniform
from mytraj import MyTraj 
import time
from scipy import integrate
from numpy import isnan, pi, isinf
from numpy.random import normal
import pandas as pd
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score
import joblib
#from trajectory import Trajectory
import json
import math

# In order to use modified modules without restarting
%load_ext autoreload
%autoreload 2

# Wind Model and Helper Functions

In [2]:
def wind_shear(alt,r,p,y,vx,vy,vz,wind_direct):
    e2b = np.array([[cos(r)*cos(y), sin(p)*sin(r)*cos(y)-cos(p)*sin(y), cos(p)*sin(r)*cos(y)+sin(p)*sin(y)],
                   [cos(r)*sin(y), sin(p)*sin(r)*sin(y)+cos(p)*cos(y), cos(p)*sin(r)*sin(y)-sin(p)*cos(y)],
                   [-sin(r), sin(p)*cos(r), cos(p)*cos(r)]])
    w6 = 0.02 #reference height wind speed
    z0 = 0.6#0.045 if cat=="C" else 0.6
    tol = 1e-3
    if alt <= tol:
        alt = tol
    W = [cos(wind_direct),sin(wind_direct),0]
    b = np.log(abs(alt)/z0)/np.log(20/z0) #reference height/z0???
    u = b*(np.dot(-w6,W))
    uw = np.dot(e2b,u)
#     uw = [gauss(0,vx*0.005),gauss(0,vy*0.005),gauss(0,vz*0.005)]
    return uw

def cartesian_to_spherical(state, waypoint_world):
    x_gate, y_gate, z_gate, yaw_gate = waypoint_world
    x_diff = -state[0] + x_gate
    y_diff = -state[1] + y_gate
    z_diff = -state[2] + z_gate

    waypoint_body = world_to_body(state, np.array([x_diff, y_diff, z_diff]))

    r = math.sqrt(waypoint_body[0]**2 + waypoint_body[1]**2 + waypoint_body[2]**2)
    phi = math.atan2(waypoint_body[1], waypoint_body[0])
    theta = math.acos(waypoint_body[2] / r)                          
    return r, phi, theta
    
def spherical_to_cartesian(state, waypoint_body):
    r, phi, theta = waypoint_body 
    x = r*sin(theta)*cos(phi)
    y = r*sin(theta)*sin(phi)
    z = r*cos(theta)
    
    waypoint_world = body_to_world(state, np.array([x,y,z]))
    x_gate, y_gate, z_gate = state[0]+waypoint_world[0], state[1]+waypoint_world[1], state[2]+waypoint_world[2]
    
    return x_gate, y_gate, z_gate


def world_to_body(state, waypoint_world):
    roll, pitch, yaw = state[3], state[4], state[5]
    alpha, beta, gamma = yaw, pitch, roll
    R = np.array([[cos(alpha)*cos(beta), cos(alpha)*sin(beta)*sin(gamma)-sin(alpha)*cos(gamma), cos(alpha)*sin(beta)*cos(gamma)+sin(alpha)*sin(gamma)],
                  [sin(alpha)*cos(beta), sin(alpha)*sin(beta)*sin(gamma)+cos(alpha)*cos(gamma), sin(alpha)*sin(beta)*cos(gamma)-cos(alpha)*sin(gamma)],
                  [-sin(beta), cos(beta)*sin(gamma), cos(beta)*cos(gamma)]])
    
    waypoint_body = np.dot(R.T, waypoint_world.reshape(-1,1))
    
    return waypoint_body.ravel()


def body_to_world(state, waypoint_body):
    roll, pitch, yaw = state[3], state[4], state[5]
    alpha, beta, gamma = yaw, pitch, roll
    R = np.array([[cos(alpha)*cos(beta), cos(alpha)*sin(beta)*sin(gamma)-sin(alpha)*cos(gamma), cos(alpha)*sin(beta)*cos(gamma)+sin(alpha)*sin(gamma)],
                  [sin(alpha)*cos(beta), sin(alpha)*sin(beta)*sin(gamma)+cos(alpha)*cos(gamma), sin(alpha)*sin(beta)*cos(gamma)-cos(alpha)*sin(gamma)],
                  [-sin(beta), cos(beta)*sin(gamma), cos(beta)*cos(gamma)]])
    
    waypoint_world = np.dot(R, waypoint_body.reshape(-1,1))
    
    return waypoint_world.ravel()

# Model Dynamics and Controller

In [3]:
def model_parameters():
    g = 9.81
    m = 1.52
    Ixx, Iyy, Izz = 0.0347563, 0.0458929, 0.0977
    I1 = (Iyy - Izz) / Ixx
    I2 = (Izz - Ixx) / Iyy
    I3 = (Ixx - Iyy) / Izz
    Jr = 0.0001
    l = 0.09
    b = 8.54858e-6
    d = 1.6e-2
    
    return g, m, Ixx, Iyy, Izz, I1, I2, I3, Jr, l, b, d

def model_dynamics(t, state, U):
    g, m, Ixx, Iyy, Izz, I1, I2, I3, Jr, l, b, d = model_parameters()
    #states: [x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot]
        
    x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot = state
    U1, U2, U3, U4 = U
    
    if (d*U1 - 2*d*U3 - b*U4) < 0:
        omega1 = - np.sqrt(- d*U1 + 2*d*U3 + b*U4) / (2*np.sqrt(b*d))
    else:
        omega1 = - np.sqrt(d*U1 - 2*d*U3 - b*U4) / (2*np.sqrt(b*d))
        
    if (d*U1 - 2*d*U2 + b*U4) < 0:
        omega2 = -np.sqrt(-d*U1 + 2*d*U2 - b*U4) / (2*np.sqrt(b*d))
    else:
        omega2 = -np.sqrt(d*U1 - 2*d*U2 + b*U4) / (2*np.sqrt(b*d))
    
    
    if (d*U1 + 2*d*U3 - b*U4) < 0:
        omega3 = -np.sqrt(-d*U1 - 2*d*U3 + b*U4) / (2*np.sqrt(b*d))
    else:
        omega3 = -np.sqrt(d*U1 + 2*d*U3 - b*U4) / (2*np.sqrt(b*d))
        
    if (d*U1 + 2*d*U2 + b*U4) < 0:
        omega4 = -np.sqrt(-d*U1 - 2*d*U2 - b*U4) / (2*np.sqrt(b*d))
    else:
        omega4 = -np.sqrt(d*U1 + 2*d*U2 + b*U4) / (2*np.sqrt(b*d))
        
    
    omega = (omega2 + omega4 - omega1 - omega3)
    
    state_dot = np.zeros(12)
    state_dot[0] = x_dot
    state_dot[1] = y_dot
    state_dot[2] = z_dot
    state_dot[3] = phi_dot
    state_dot[4] = theta_dot
    state_dot[5] = psi_dot
    
    state_dot[6] = (cos(phi)*sin(theta)*cos(psi) + sin(phi)*sin(psi))*U[0]/m
    state_dot[7] = (cos(phi)*sin(theta)*sin(psi) - sin(phi)*cos(psi))*U[0]/m
    state_dot[8] = -g + cos(phi)*cos(theta)*U[0]/m    
    state_dot[9] = theta_dot*psi_dot*I1 - Jr / Ixx * theta_dot * omega  + l/Ixx*U[1]
    state_dot[10] = phi_dot*psi_dot*I2 + Jr / Iyy * phi_dot * omega + l/Iyy*U[2]
    state_dot[11] = phi_dot*theta_dot*I3 + 1/Izz*U[3]
    
    return state_dot
    
def backstepping(A1, A2, A3, A4, A5, A6, U_list, ref_traj, states):
    g, m, Ixx, Iyy, Izz, I1, I2, I3, Jr, l, b, d = model_parameters()
    
    U1, U2, U3, U4 = U_list
    
    #states: [x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot]
    x, y, z = states[0], states[1], states[2]
    phi, theta, psi = states[3], states[4], states[5]
    x_dot, y_dot, z_dot = states[6], states[7], states[8]
    phi_dot, theta_dot, psi_dot = states[9], states[10], states[11]
    
#     ref_traj = [xd[i], yd[i], zd[i], xd_dot[i], yd_dot[i], zd_dot[i], 
#                 xd_ddot[i], yd_ddot[i], zd_ddot[i], xd_dddot[i], yd_dddot[i],
#                 xd_ddddot[i], yd_ddddot[i], psid[i], psid_dot[i], psid_ddot[i]]

    
    xd, yd, zd = ref_traj[0], ref_traj[1], ref_traj[2], 
    xd_dot, yd_dot, zd_dot = ref_traj[3], ref_traj[4], ref_traj[5]
    xd_ddot, yd_ddot, zd_ddot = ref_traj[6], ref_traj[7], ref_traj[8]
    xd_dddot, yd_dddot = ref_traj[9], ref_traj[10]
    xd_ddddot, yd_ddddot = ref_traj[11], ref_traj[12]
    psid, psid_dot, psid_ddot = ref_traj[13], ref_traj[14], ref_traj[15]
    
    x1, x2, x3 = array([[x], [y]]), array([[x_dot], [y_dot]]), array([[phi], [theta]])
    x4, x5, x6 = array([[phi_dot], [theta_dot]]), array([[psi], [z]]), array([[psi_dot], [z_dot]])
    
    g0 = array([[np.cos(psi), np.sin(psi)],  [np.sin(psi), -np.cos(psi)]])
    g0_inv = array([[np.cos(psi), np.sin(psi)],  [np.sin(psi), -np.cos(psi)]])
    
    g1 = array([[theta_dot*psi_dot*I1],  [phi_dot*psi_dot*I2]])
    g2 = array([[phi_dot*theta_dot*I3],  [-g]])
    
    l0 = array([[np.cos(phi)*np.sin(theta)],  [np.sin(phi)]])*U1/m 
    dl0_dx3 = array([[-np.sin(phi)*np.sin(theta), np.cos(phi)*np.cos(theta)],  [np.cos(phi), 0]])*U1/m 
    dl0_dx3_inv = array([[0, 1/np.cos(phi)],  [1/np.cos(theta)*1/np.cos(phi), 1/np.cos(phi)*np.tan(theta)*np.tan(phi)]])*m/U1 
    dl0_dx3_inv_dot = array([[0, 1/np.cos(phi)*np.tan(phi)*phi_dot], 
                             [1/np.cos(theta)*1/np.cos(phi)*(np.tan(theta)*theta_dot + np.tan(phi)*phi_dot), 1/np.cos(phi)*((1/np.cos(theta))**2*np.tan(phi)*theta_dot + (-1+2*(1/np.cos(phi))**2)*np.tan(theta)*phi_dot)]])*m/U1 
                
#     Omega_square = Omega_coef_inv * abs([U1/b  U2/b  U3/b  U4/d]) 
#     Omega_param = sqrt(Omega_square) 
#     omega = Omega_param(2) + Omega_param[3] - Omega_param(1) - Omega_param(3) 

#     h1 = [-Jr/Ixx*theta_dot*omega  Jr/Iyy*phi_dot*omega] 
    h1 = 0 
    k1 = diag([l/Ixx, l/Iyy]) 
    k1_inv = diag([Ixx/l, Iyy/l]) 
    k2 = diag([1/Izz, np.cos(phi)*np.cos(theta)/m]) 
    k2_inv = diag([Izz, m/(np.cos(phi)*np.cos(theta))]) 
    
    x1d = array([[xd], [yd]])  
    x1d_dot = array([[xd_dot], [yd_dot]]) 
    x1d_ddot = array([[xd_ddot], [yd_ddot]]) 
    x1d_dddot = array([[xd_dddot], [yd_dddot]]) 
    x1d_ddddot = array([[xd_ddddot], [yd_ddddot]]) 
    
    x5d = array([[psid], [zd]])
    x5d_dot = array([[psid_dot], [zd_dot]]) 
    x5d_ddot = array([[psid_ddot], [zd_ddot]]) 
    
    z1 = x1d - x1 
    v1 = x1d_dot + dot(A1,z1) 
    z2 = v1 - x2 
    z1_dot = -dot(A1,z1) + z2 
    v1_dot = x1d_ddot + dot(A1,z1_dot) 
    v2 = dot(g0_inv, z1 + v1_dot + dot(A2,z2)) 
    z3 = v2 - l0  
    z2_dot = -z1 - dot(A2,z2) + dot(g0,z3) 
    z1_ddot = -dot(A1,z1_dot) + z2_dot 
    v1_ddot = x1d_dddot + dot(A1, z1_ddot) 
    v2_dot = dot(g0_inv, z1_dot + v1_ddot + dot(A2,z2_dot)) 
    v3 = dot(dl0_dx3_inv, dot(g0.T,z2) + v2_dot + dot(A3, z3)) 
    z4 = v3 - x4 
    z3_dot = -dot(g0.T, z2) - dot(A3,z3) + dot(dl0_dx3, z4) 
    z2_ddot = - z1_dot - dot(A2, z2_dot) + dot(g0, z3_dot) 
    z1_dddot = -dot(A1, z1_ddot) + z2_ddot 
    v1_dddot = x1d_ddddot + dot(A1, z1_dddot) 
    v2_ddot = dot(g0_inv, z1_ddot + v1_dddot + dot(A2, z2_ddot)) 
    v3_dot = dot(dl0_dx3_inv, dot(g0.T, z2_dot) + v2_ddot + dot(A3, z3_dot)) + dot(dl0_dx3_inv_dot, dot(g0.T, z2) + v2_dot + dot(A3, z3))
    l1 = dot(k1_inv, dot(dl0_dx3.T, z3) + v3_dot - g1 - h1 + dot(A4, z4)).ravel()
    
    z5 = x5d - x5 
    v5 = x5d_dot + dot(A5, z5) 
    z6 = v5 - x6 
    z5_dot = - dot(A5, z5) + z6 
    v5_dot = x5d_ddot + dot(A5, z5_dot) 
    l2 = dot(k2_inv, z5 + v5_dot - g2 + dot(A6, z6)).ravel()
    
    U1, U2, U3, U4 = l2[1], l1[0], l1[1], l2[0]
    
    U1 = np.clip(U1, 1.0, 1e2)
    U2 = np.clip(U2, -1e3, 1e3)
    U3 = np.clip(U3, -1e3, 1e3)
    U4 = np.clip(U4, -1e2, 1e2)
    
    U = np.array([U1, U2, U3, U4])
    
    return U

def get_control_input(cont, Controllers, U0, current_traj, state):
    #single_shot = ["Back"] # just a check flight to test the coefficients
    if (cont == Controllers[0]): #Backstepping_1
        A1, A2, A3 = 15*diag([1,1]), 10*diag([1,1]), 15*diag([1,1]) 
        A4, A5, A6 = 10*diag([1,1]), 15*diag([1,1]), 10*diag([1,1]) 
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state) 
    elif (cont == Controllers[1]): #Backstepping_2
        A1, A2, A3 = 10*diag([1,1]), 5*diag([1,1]), 10*diag([1,1]) 
        A4, A5, A6 = 5*diag([1,1]), 10*diag([1,1]), 5*diag([1,1])
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state) 
    elif (cont == Controllers[2]): #Backstepping_3
        A1, A2, A3 = 5*diag([1,1]), 3*diag([1,1]), 10*diag([1,1]) 
        A4, A5, A6 = 7*diag([1,1]), 1*diag([1,1]), 1*diag([1,1])  
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state)
    elif (cont == Controllers[3]): #Backstepping_4
        A1, A2, A3 = 2*diag([1,1]), 5*diag([1,1]), 2*diag([1,1]) 
        A4, A5, A6 = 5*diag([1,1]), 2*diag([1,1]), 5*diag([1,1]) 
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state)
    elif (cont == single_shot[0]): #Single Shot
        A1, A2, A3 = 1*diag([1,1]), 1*diag([1,1]), 1*diag([1,1]) 
        A4, A5, A6 = 1*diag([1,1]), 1*diag([1,1]), 1*diag([1,1]) 
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state)
    return U

# Neural Network

In [4]:
def predict(X, model, device, method="MAX"):
    softmax = nn.Softmax(dim=1)
    inputs = torch.from_numpy(X).to(device)
    outputs = model(inputs.float())
 
    if method=="MAX":
        _, pred = torch.max(outputs, 1)
        return pred[0]
    elif method=="DICE":
        probs = softmax(outputs).cpu().detach().numpy()[0]
        pred_index = np.random.choice([0, 1, 2, 3], 1, p=probs)[0]
        return pred_index
    elif method=="MIX":
        probs = softmax(outputs).cpu().detach().numpy()[0]     
        return probs
        
    
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop_layer = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(40, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc2(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc3(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc4(x))
        x = self.drop_layer(x)
        x = self.fc5(x)
        return x

In [5]:
def write_stats(stats_columns, stats, filename): #testno,stats_columns
    df_stats = pd.DataFrame([stats], columns=stats_columns)
    df_stats.to_csv(filename, mode='a', index=False,header=not os.path.isfile(filename))
    
def quad_sim(Tf, state0, ref_traj, std_list, scaler=None, model=None, device=None, status="TEST", method="MAX"):
#     states: [x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot]

    N = ref_traj.shape[0]
    costValue = 0
    U = [1,0,0,0]

    Controllers = ["Backstepping_1", "Backstepping_2", "Backstepping_3", "Backstepping_4"]
    single_shot = ["Back"] # just a check flight to test the coefficients
    count_dict = {"Backstepping_1": 0, "Backstepping_2": 0, "Backstepping_3": 0, "Backstepping_4": 0}
    U_dict = {"Backstepping_1": 0, "Backstepping_2": 0, "Backstepping_3": 0, "Backstepping_4": 0}
    state_list = {"MAX":[], "MIX":[], "DICE":[]}#, "DT":[], "FOREST":[], "Backstepping_1":[], "Backstepping_2":[], "Backstepping_3":[], "Backstepping_4":[]}
    state = np.copy(state0)
    r_std, phi_std, theta_std, psi_std = std_list
    
    for i in range(N):
        
        #     ref_traj = [xd[i], yd[i], zd[i], xd_dot[i], yd_dot[i], zd_dot[i], 
        #                 xd_ddot[i], yd_ddot[i], zd_ddot[i], xd_dddot[i], yd_dddot[i],
        #                 xd_ddddot[i], yd_ddddot[i], psid[i], psid_dot[i], psid_ddot[i]]
        prev_traj_index = np.maximum(0, i-1)
        
        current_traj = [ref_traj[i][0], ref_traj[i][1], ref_traj[i][2], ref_traj[i][3], ref_traj[i][4], ref_traj[i][5], 
                        ref_traj[i][6], ref_traj[i][7], ref_traj[i][8], ref_traj[i][9], ref_traj[i][10], ref_traj[i][11],
                        ref_traj[i][12], ref_traj[i][13], ref_traj[i][14], ref_traj[i][15]]
        
        prev_traj = [ref_traj[prev_traj_index][0], ref_traj[prev_traj_index][1], ref_traj[prev_traj_index][2], ref_traj[prev_traj_index][3], ref_traj[prev_traj_index][4], ref_traj[prev_traj_index][5], 
                     ref_traj[prev_traj_index][6], ref_traj[prev_traj_index][7], ref_traj[prev_traj_index][8], ref_traj[prev_traj_index][9], ref_traj[prev_traj_index][10], ref_traj[prev_traj_index][11],
                     ref_traj[prev_traj_index][12], ref_traj[prev_traj_index][13], ref_traj[prev_traj_index][14], ref_traj[prev_traj_index][15]]

        U0 = np.copy(U)
        
        t_current = Tf / N * i
        time_rate = float(t_current / Tf)
        Upr_abs_sum = np.sum(np.abs(U0))
        
#         feature_names = ['x0', 'y0', 'z0', 'x_dot0','y_dot0','z_dot0', 'phi0','theta0','yaw0', 'phi_dot0','theta_dot0','yaw_dot0', 
#                  'xf', 'yf', 'zf', 'x_dotf','y_dotf','z_dotf','x_ddotf','y_ddotf','z_ddotf',
#                  'pos_diffx','pos_diffy','pos_diffz','time_rate','t', 'Tf', 
#                  'xp', 'yp', 'zp', 'x_dotp','y_dotp','z_dotp','x_ddotp','y_ddotp','z_ddotp', 'u_abs_p',
#                  'r_std', 'phi_std', 'psi_std']
        
        #WIND EFFECT    
#         wind_direct = (np.random.randint(1,360)/180)*pi #Deg2Rad
#         wind_noisev = wind_shear(state[2],state[3],state[4],state[5],state[6],state[7],state[8],wind_direct)
        
        #Test part
        if status == "TEST":
            ## ADD DISTURBANCE ##
#             state[6] += wind_noisev[0]
#             state[7] += wind_noisev[1]
#             state[8] += wind_noisev[2]

            ## ADD NOISE ##
            state[6] = normal(state[6], r_std / 3.0)
            state[7] = normal(state[7], r_std / 3.0)
            state[8] = normal(state[8], r_std / 3.0)
            state[9] = normal(state[9], phi_std)
            state[10] = normal(state[10], theta_std)
            state[11] = normal(state[11], psi_std)

            X_test = np.array([state[0], state[1], state[2], state[6], state[7], state[8], state[3], state[4], state[5], state[9], state[10], state[11],
                        current_traj[0], current_traj[1], current_traj[2], current_traj[3], current_traj[4], current_traj[5], current_traj[6], current_traj[7], current_traj[8],
                        state[0]-current_traj[0], state[1]-current_traj[1], state[2]-current_traj[2], time_rate, t_current, Tf, 
                        prev_traj[0], prev_traj[1], prev_traj[2], prev_traj[3], prev_traj[4], prev_traj[5], prev_traj[6], prev_traj[7], prev_traj[8], 
                        Upr_abs_sum, r_std, phi_std, theta_std, psi_std]).reshape(1,-1)
    
#             X_test = scaler.transform(X_test)

            
            state_list[method].append(state)
        
            if method == "MAX":
                pred = predict(X_test, model, device, method)
                cont = Controllers[pred]
                U = get_control_input(cont, Controllers, U0, current_traj, state)
                count_dict[cont] += 1
                
            elif method == "DICE":
                pred = predict(X_test, model, device, method)
                cont = Controllers[pred]
                U = get_control_input(cont, Controllers, U0, current_traj, state)
                count_dict[cont] += 1
                
            elif method == "MIX":
                probs = predict(X_test, model, device, method)
                U = 0
                for k in range(4):
                    cont = Controllers[k]
                    cont_prob = probs[k]
                    U_single = get_control_input(cont, Controllers, U0, current_traj, state)
                    U = U + (cont_prob * U_single)
                    
            elif method == "DT":
                pred = decision_tree.predict(X_test)[0]
                cont = Controllers[pred]
                U = get_control_input(cont, Controllers, U0, current_traj, state)
                count_dict[cont] += 1
                
            elif method == "FOREST":
                pred = random_forest.predict(X_test)[0]
                cont = Controllers[pred]
                U = get_control_input(cont, Controllers, U0, current_traj, state)
                count_dict[cont] += 1
            else:
                #This time a controller info comes in method variable
                U = get_control_input(method, Controllers, U0, current_traj, state)
                
            sol = integrate.solve_ivp(fun=model_dynamics, t_span=(0, dtau), y0=state, args = (U,))
            state = sol.y[:,-1]
            
            
            if (np.abs(state[3]) > pi/2)  | (np.abs(state[4]) > pi/2):
                costValue = 1e6
                method_cost_list[method].append(costValue)
                break
            else:
                position_tracking_error = np.power((current_traj[0]-state[0]),2) + np.power((current_traj[1]-state[1]),2) + np.power((current_traj[2]-state[2]),2)
                velocity_tracking_error = np.power((current_traj[3]-state[6]),2) + np.power((current_traj[4]-state[7]),2) + np.power((current_traj[5]-state[8]),2)
                angular_error = state[3]**2 + state[4]**2 + np.power((current_traj[13]-state[5]),2)
                cont_input = U[0]**2 + U[1]**2 + U[2]**2 + U[3]**2
                costValue = costValue + (coeff_pos*position_tracking_error + 
                                         coeff_vel*velocity_tracking_error +
                                         coeff_angle*angular_error + coeff_control*cont_input)
                
            
            
        elif status == "DATA_COLLECTION":
            cost_dict = {"Backstepping_1": 0, "Backstepping_2": 0, "Backstepping_3": 0, "Backstepping_4": 0}
            state_dict = {"Backstepping_1": [], "Backstepping_2": [], "Backstepping_3": [], "Backstepping_4": []}
            
            noisy_state = np.copy(state)          
#             ## ADD DISTURBANCE ##
#             noisy_state[6] += wind_noisev[0]
#             noisy_state[7] += wind_noisev[1]
#             noisy_state[8] += wind_noisev[2]
                
            ## ADD NOISE ##
            noisy_state[6] = normal(state[6], r_std / 3.0)
            noisy_state[7] = normal(state[7], r_std / 3.0)
            noisy_state[8] = normal(state[8], r_std / 3.0)
            noisy_state[9] = normal(state[9], phi_std)
            noisy_state[10] = normal(state[10], theta_std)
            noisy_state[11] = normal(state[11], psi_std)
            
            for cont in Controllers:  
                U = get_control_input(cont, Controllers, U0, current_traj, state)
                
                sol = integrate.solve_ivp(fun=model_dynamics, t_span=(0, dtau), y0=noisy_state, args = (U,))
                new_state = sol.y[:,-1]
                
                state_dict[cont] = new_state
                U_dict[cont] = U
                
                if (np.abs(new_state[3]) > pi/2)  | (np.abs(new_state[4]) > pi/2):
                    costValue = 1e12
                    cost_dict[cont] = costValue
                    continue

            #     states: [x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot]

                position_tracking_error = np.power((current_traj[0]-new_state[0]),2) + np.power((current_traj[1]-new_state[1]),2) + np.power((current_traj[2]-new_state[2]),2)
                velocity_tracking_error = np.power((current_traj[3]-new_state[6]),2) + np.power((current_traj[4]-new_state[7]),2) + np.power((current_traj[5]-new_state[8]),2)
                angular_error = new_state[3]**2 + new_state[4]**2 + np.power((current_traj[13]-new_state[5]),2)
                cont_input = U[0]**2 + U[1]**2 + U[2]**2 + U[3]**2
                costValue = (coeff_pos*position_tracking_error + coeff_vel*velocity_tracking_error +
                             coeff_angle*angular_error + coeff_control*cont_input)
                cost_dict[cont] = costValue
                
                
            min_cost_index = min(cost_dict.items(), key=lambda x: x[1])[0]
            count_dict[min_cost_index] += 1   
            U = U_dict[min_cost_index]
            
            state = state_dict[min_cost_index]
            
            if(i%20==0):
                write_stats(flight_columns,
                        [state[0], state[1], state[2], state[6], state[7], state[8], state[3], state[4], state[5], state[9], state[10], state[11],
                        current_traj[0], current_traj[1], current_traj[2], current_traj[3], current_traj[4], current_traj[5], current_traj[6], current_traj[7], current_traj[8],
                        state[0]-current_traj[0], state[1]-current_traj[1], state[2]-current_traj[2], time_rate, t_current, Tf, 
                        prev_traj[0], prev_traj[1], prev_traj[2], prev_traj[3], prev_traj[4], prev_traj[5], prev_traj[6], prev_traj[7], prev_traj[8], 
                        Upr_abs_sum, r_std, phi_std, theta_std, psi_std, min_cost_index], flight_filename)
            
        elif status == "SINGLE_SHOT":
#             ## ADD DISTURBANCE ##
#             noisy_state[6] += wind_noisev[0]
#             noisy_state[7] += wind_noisev[1]
#             noisy_state[8] += wind_noisev[2]
                
            ## ADD NOISE ##
            state[6] = normal(state[6], r_std / 3.0)
            state[7] = normal(state[7], r_std / 3.0)
            state[8] = normal(state[8], r_std / 3.0)
            state[9] = normal(state[9], phi_std)
            state[10] = normal(state[10], theta_std)
            state[11] = normal(state[11], psi_std)
            
            U = get_control_input(single_shot[0],Controllers, U0, current_traj, state)
            sol = integrate.solve_ivp(fun=model_dynamics, t_span=(0, dtau), y0=state, args = (U,))
            state = sol.y[:,-1] 
            

            if (np.abs(state[3]) > pi/2)  | (np.abs(state[4]) > pi/2):
                costValue = 1e12
                break

            position_tracking_error = np.power((current_traj[0]-state[0]),2) + np.power((current_traj[1]-state[1]),2) + np.power((current_traj[2]-state[2]),2)
            velocity_tracking_error = np.power((current_traj[3]-state[6]),2) + np.power((current_traj[4]-state[7]),2) + np.power((current_traj[5]-state[8]),2)
            angular_error = state[3]**2 + state[4]**2 + np.power((current_traj[13]-state[5]),2)
            cont_input = U[0]**2 + U[1]**2 + U[2]**2 + U[3]**2
            costValue = costValue + (coeff_pos*position_tracking_error + 
                                     coeff_vel*velocity_tracking_error +
                                     coeff_angle*angular_error + coeff_control*cont_input)
            
        

    if (status == "TEST" or status == "SINGLE_SHOT"):
        print ("Cost Value: ", costValue)
        method_cost_list[method].append(costValue)
    if (method == "MAX" or method == "DICE" or method == "DT" or method == "FOREST"):
        total_call = np.array([0,0,0,0])
        print ("How many times Backstepping_1 is called?: ", count_dict["Backstepping_1"])
        print ("How many times Backstepping_2 is called?: ", count_dict["Backstepping_2"])
        print ("How many times Backstepping_3 is called?: ", count_dict["Backstepping_3"])
        print ("How many times Backstepping_4 is called?: ", count_dict["Backstepping_4"])
        total_call = np.array([count_dict["Backstepping_1"],count_dict["Backstepping_2"],count_dict["Backstepping_3"],count_dict["Backstepping_4"]])
    print ("Final state, x: {0:.3}, y: {1:.3}, z: {2:.3}, phi: {3:.3}, theta: {4:.3}, psi: {5:.3}, vx: {6:.3}, vy: {7:.3}, vz: {8:.3}, p: {9:.3}, q: {10:.3}, r: {11:.3}".format(state[0],state[1],state[2],state[3],state[4],state[5], state[6],state[7],state[8], state[9],state[10],state[11]))
    
    if (status == "TEST"):
        return state_list
    return total_call

## Simulation of Drone

In [6]:
uniform(low=0.0, high=5.0)

2.4571521205613482

In [12]:
flight_columns = ['x0', 'y0', 'z0', 'x_dot0','y_dot0','z_dot0', 'phi0','theta0','yaw0', 'phi_dot0','theta_dot0','yaw_dot0', 
                 'xf', 'yf', 'zf', 'x_dotf','y_dotf','z_dotf','x_ddotf','y_ddotf','z_ddotf',
                 'pos_diffx','pos_diffy','pos_diffz','time_rate','t', 'Tf', 
                 'xp', 'yp', 'zp', 'x_dotp','y_dotp','z_dotp','x_ddotp','y_ddotp','z_ddotp', 'u_abs_p', 
                 'r_std', 'phi_std', 'theta_std', 'psi_std', 'controller_ID']

cost_columns = ["MAX", "MIX", "DICE", "DT", "FOREST", "Backstepping_1", "Backstepping_2", "Backstepping_3", "Backstepping_4"]
flight_filename = "flight_gauss.csv"
cost_filename = 'method_cost_list.csv'

STATUS = "DATA_COLLECTION"
# STATUS = "SINGLE_SHOT"
# STATUS = "TEST"
METHOD_LIST = ["MAX", "MIX", "DICE"]#, "DT", "FOREST", "Backstepping_1", "Backstepping_2", "Backstepping_3", "Backstepping_4"]
method_cost_list = {"MAX":[], "MIX":[], "DICE":[], "DT":[], "FOREST":[], "Backstepping_1":[], "Backstepping_2":[], "Backstepping_3":[], "Backstepping_4":[]}
dtau = 1e-3

if STATUS == "TEST":        
    #Decision Tree classifier
    decision_tree = joblib.load('Classifier/decision_tree.sav')
    #To normalize data
    scaler = joblib.load('Classifier/std_scaler.bin')

    #Neural network
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Net()
    model = model.to(device)
    model.load_state_dict(torch.load('Classifier/best_model.pt'))
    model.eval()


K = 25 # how many different trajectories to be followed

v_lim = 0.5
acc_lim = 0.1
ang_lim = 45*np.pi/180
ang_vel_lim = 0.5
pos_lim = 15



U_list = []
all_states = []

Rvalues = np.loadtxt('Rvalues.txt', delimiter =' ', usecols =(0, 1, 2, 3), unpack = True)
Rvalues = Rvalues.T
r_std = np.sqrt(np.mean(Rvalues[:,0]))
phi_std = np.sqrt(np.mean(Rvalues[:,1]))
theta_std = np.sqrt(np.mean(Rvalues[:,2]))
psi_std = np.sqrt(np.mean(Rvalues[:,3]))


#In case you specify the waypoints instead of sampling them
#waypoint is formed of x,y,z,yaw respectively
# waypoint_list = np.array([[6.,4.,2.,pi/2.],
#                          [12.,8.,4.,pi/4.],
#                          [15.,2.,6.,-pi/3.],
#                          [6.,9.,12.,-pi/4.]])
# Tf_list = [5.,4.5,3.5, 4.]


std_list = 0.25*np.array([r_std, phi_std, theta_std, psi_std])
coeff_angle = 0.85
coeff_pos = 1.0
coeff_vel = 0.1
coeff_control = 0


print ("RUN MODE: ", STATUS)

complete_calls = np.array([0,0,0,0])

for j in range(K):
    pos0 = [uniform(low=-2.0, high=2.0), uniform(low=-2.0, high=2.0), uniform(low=0.0, high=5.0)]
    vel0 = [uniform(low=-v_lim, high=v_lim), uniform(low=-v_lim, high=v_lim), uniform(low=-v_lim, high=v_lim)]
    acc0 = [uniform(low=-acc_lim, high=acc_lim), uniform(low=-acc_lim, high=acc_lim), uniform(low=-acc_lim, high=acc_lim)]
    ang0 = [uniform(low=-ang_lim, high=ang_lim), uniform(low=-ang_lim, high=ang_lim), uniform(low=-pi, high=pi)]
    ang_vel0 = [uniform(low=-ang_vel_lim, high=ang_vel_lim), uniform(low=-ang_vel_lim, high=ang_vel_lim), uniform(low=-ang_vel_lim, high=ang_vel_lim)]
    posf = [uniform(low=-pos_lim, high=pos_lim), uniform(low=-pos_lim, high=pos_lim), uniform(low=0.0, high=1.5*pos_lim)]
    yawf = uniform(low=-pi, high=pi)
    velf = [0.,0.,0.]#[uniform(low=0, high=0.5), uniform(low=0, high=0.5), uniform(low=0, high=0.5)]
    accf = [0.,0.,0.]#[uniform(low=0, high=0.15), uniform(low=0, high=0.15), uniform(low=0, high=0.15)]
    Tf = uniform(low=5, high=15)


    print ("-"*120)
    
    state0 = [pos0[0], pos0[1], pos0[2], ang0[0], ang0[1], ang0[2], vel0[0], vel0[1], vel0[2], ang_vel0[0], ang_vel0[1], ang_vel0[1]]
    
    
    x_gate, y_gate, z_gate, yaw_gate = posf[0], posf[1], posf[2], yawf
    waypoint_world = np.array([x_gate, y_gate, z_gate, yaw_gate])
    waypoint_body = cartesian_to_spherical(state0, waypoint_world)
    waypoint_body_noisy = np.array([normal(waypoint_body[0], r_std), 
                                    normal(waypoint_body[1], phi_std), normal(waypoint_body[2], theta_std)])
    waypoint_world = spherical_to_cartesian(state0, waypoint_body_noisy)

    posf = [waypoint_world[0], waypoint_world[1], waypoint_world[2]]
    yawf = normal(yaw_gate, psi_std)
    
    
    

    
    print ("Ground truth values of gate")
    print ("x_gate={0:.4}, y_gate={1:.4}, z_gate={2:.4}, psi_gate={3:.4}".format(x_gate, y_gate, z_gate, yaw_gate))
    print ("r={0:.4}, phi={1:.4}, theta={2:.4}".format(waypoint_body[0], waypoint_body[1], waypoint_body[2]))
    print ("r_noisy={0:.4}, phi_noisy={1:.4}, theta_noisy={2:.4}".format(waypoint_body_noisy[0], waypoint_body_noisy[1], waypoint_body_noisy[2]))
    print ("Measured values of gate")
    print ("x_gate={0:.4}, y_gate={1:.4}, z_gate={2:.4}, psi_gate={3:.4}".format(waypoint_world[0], waypoint_world[1], waypoint_world[2], yawf))
    print ("")
    print ("Init, x: {0:.3}, y: {1:.3}, z: {2:.3}, phi: {3:.3}, theta: {4:.3}, psi: {5:.3}, vx: {6:.3}, vy: {7:.3}, vz: {8:.3}, p: {9:.3}, q: {10:.3}, r: {11:.3}"
           .format(state0[0], state0[1], state0[2], state0[3], state0[4], state0[5], state0[6], state0[7], state0[8], state0[9], state0[10], state0[11]))
    print ("Goal, x: {0:.3}, y: {1:.3}, z: {2:.3}, yaw: {3:.3} in {4:.3} s."
           .format(posf[0], posf[1], posf[2], yawf, Tf))
    
    
    
    traj = MyTraj(gravity = -9.81)
    trajectory = traj.givemetraj(pos0, vel0, acc0, posf, velf, accf, Tf)
    
    N = int(Tf/dtau)
    t = linspace(0,Tf,num = N)
    
    xd, yd, zd, psid = zeros(t.shape), zeros(t.shape), zeros(t.shape), zeros(t.shape)
    xd_dot, yd_dot, zd_dot, psid_dot = zeros(t.shape), zeros(t.shape), zeros(t.shape), zeros(t.shape)
    xd_ddot, yd_ddot, zd_ddot, psid_ddot = zeros(t.shape), zeros(t.shape), zeros(t.shape), zeros(t.shape)
    xd_dddot, yd_dddot, zd_dddot = zeros(t.shape), zeros(t.shape), zeros(t.shape)
    xd_ddddot, yd_ddddot, zd_ddddot = zeros(t.shape), zeros(t.shape), zeros(t.shape)
    
    i = 0
    ts = 0
    
    for i in range(N):
        pos_des, vel_des, acc_des = traj.givemepoint(trajectory, ts)
        
        xd[i], yd[i], zd[i] = pos_des[0], pos_des[1], pos_des[2]
        xd_dot[i], yd_dot[i], zd_dot[i] = vel_des[0], vel_des[1], vel_des[2]
        xd_ddot[i], yd_ddot[i], zd_ddot[i] = acc_des[0], acc_des[1], acc_des[2]
        psid[i] = yawf
        ts += dtau
    
    
    ref_traj = np.c_[xd, yd, zd, xd_dot, yd_dot, zd_dot, xd_ddot, yd_ddot, zd_ddot,
                     xd_dddot, yd_dddot, xd_ddddot, yd_ddddot,
                     psid, psid_dot, psid_ddot]
    
    
    if STATUS == "DATA_COLLECTION" or STATUS == "SINGLE_SHOT":
        toplam_cagrilma = quad_sim(Tf, state0, ref_traj, std_list, status=STATUS)
        complete_calls += toplam_cagrilma
    elif STATUS == "TEST":
        for method in METHOD_LIST:
            print ("\nTEST MODE: ", method)
            states = quad_sim(Tf, state0, ref_traj, std_list, scaler=scaler, model=model, device=device, status=STATUS, method=method)
            all_states.append(states)

        if j % 1 == 0:
            write_stats(cost_columns,
                        [method_cost_list["MAX"], method_cost_list["MIX"], method_cost_list["DICE"], method_cost_list["DT"],
                         method_cost_list["FOREST"], method_cost_list["Backstepping_1"], method_cost_list["Backstepping_2"], 
                         method_cost_list["Backstepping_3"], method_cost_list["Backstepping_4"]], cost_filename)

print("Total Calls:", complete_calls)

print("Total calls: ", np.sum(complete_calls))
print("Backstepping_1 (the most agile): %", complete_calls[0]/np.sum(complete_calls)*100)
print("Backstepping_2 (agile): %", complete_calls[1]/np.sum(complete_calls)*100)
print("Backstepping_3 (smooth): %", complete_calls[2]/np.sum(complete_calls)*100)
print("Backstepping_4 (the smoothest): %", complete_calls[3]/np.sum(complete_calls)*100)

RUN MODE:  DATA_COLLECTION
------------------------------------------------------------------------------------------------------------------------
Ground truth values of gate
x_gate=14.24, y_gate=-14.22, z_gate=4.176, psi_gate=2.054
r=20.12, phi=-0.9611, theta=2.394
r_noisy=19.83, phi_noisy=-1.002, theta_noisy=2.412
Measured values of gate
x_gate=13.68, y_gate=-14.44, z_gate=3.786, psi_gate=2.094

Init, x: -1.61, y: -1.86, z: 4.99, phi: -0.651, theta: -0.474, psi: 0.432, vx: -0.309, vy: -0.401, vz: 0.0232, p: -0.445, q: 0.136, r: 0.136
Goal, x: 13.7, y: -14.4, z: 3.79, yaw: 2.09 in 8.23 s.
How many times Backstepping_1 is called?:  4313
How many times Backstepping_2 is called?:  1585
How many times Backstepping_3 is called?:  1693
How many times Backstepping_4 is called?:  634
Final state, x: 13.6, y: -14.4, z: 3.78, phi: -0.145, theta: -0.274, psi: 2.1, vx: 0.0516, vy: -0.167, vz: -0.226, p: 0.173, q: 1.51, r: -0.126
-------------------------------------------------------------------

How many times Backstepping_1 is called?:  5733
How many times Backstepping_2 is called?:  2740
How many times Backstepping_3 is called?:  3956
How many times Backstepping_4 is called?:  1386
Final state, x: -11.2, y: -15.1, z: 13.9, phi: -0.179, theta: 0.0651, psi: -2.96, vx: -0.0359, vy: 0.498, vz: -0.0295, p: -1.63, q: -0.893, r: 0.125
------------------------------------------------------------------------------------------------------------------------
Ground truth values of gate
x_gate=2.136, y_gate=-9.724, z_gate=9.468, psi_gate=0.8962
r=10.18, phi=-2.442, theta=0.5697
r_noisy=10.32, phi_noisy=-2.448, theta_noisy=0.5738
Measured values of gate
x_gate=2.178, y_gate=-9.86, z_gate=9.495, psi_gate=0.8629

Init, x: -0.853, y: -1.59, z: 4.12, phi: -0.118, theta: -0.558, psi: 1.63, vx: -0.027, vy: 0.243, vz: -0.029, p: -0.0836, q: 0.197, r: 0.197
Goal, x: 2.18, y: -9.86, z: 9.49, yaw: 0.863 in 7.06 s.
How many times Backstepping_1 is called?:  2485
How many times Backstepping_2 is call

How many times Backstepping_1 is called?:  4518
How many times Backstepping_2 is called?:  3005
How many times Backstepping_3 is called?:  3682
How many times Backstepping_4 is called?:  1586
Final state, x: -6.36, y: -0.523, z: 3.57, phi: 0.0895, theta: 0.203, psi: -3.09, vx: 0.0662, vy: 0.0503, vz: -0.0114, p: -1.3, q: 0.958, r: 0.0322
------------------------------------------------------------------------------------------------------------------------
Ground truth values of gate
x_gate=-1.82, y_gate=0.1384, z_gate=22.2, psi_gate=1.292
r=21.32, phi=1.405, theta=0.6068
r_noisy=21.61, phi_noisy=1.415, theta_noisy=0.6062
Measured values of gate
x_gate=-1.778, y_gate=0.04786, z_gate=22.5, psi_gate=1.258

Init, x: 0.669, y: -0.445, z: 1.03, phi: 0.528, theta: 0.00146, psi: 2.27, vx: 0.00789, vy: 0.459, vz: 0.156, p: -0.377, q: 0.321, r: 0.321
Goal, x: -1.78, y: 0.0479, z: 22.5, yaw: 1.26 in 8.76 s.
How many times Backstepping_1 is called?:  3413
How many times Backstepping_2 is called?:

### Variance values in measuring distances and rotations of gates

In [ ]:
# Rvalues = np.loadtxt('Rvalues.txt', delimiter =' ', usecols =(0, 1, 2, 3), unpack = True)
# Rvalues = Rvalues.T
# r_std = np.sqrt(np.mean(Rvalues[:,0]))
# phi_std = np.sqrt(np.mean(Rvalues[:,1]))
# theta_std = np.sqrt(np.mean(Rvalues[:,2]))
# psi_std = np.sqrt(np.mean(Rvalues[:,3]))
# np.mean(np.sqrt(Rvalues[:,1]))*180/pi

## Function for getting the state list of drone in test scenario

In [ ]:
# desired_method = "DICE"

# states_list = {"MAX":[], "MIX":[], "DICE":[]}#, "DT":[], "FOREST":[], "Backstepping_1":[], "Backstepping_2":[], "Backstepping_3":[], "Backstepping_4":[]}
# index_method = {0:"MAX", 1:"MIX", 2:"DICE"}
# args = []
# N = 0
# Tf = 0
# M = len(METHOD_LIST)
# for i in range(len(METHOD_LIST)):
#     method = i
#     while True:
#         states_list[index_method[method]].append(all_states[i][index_method[method]])
#         i += M
#         if (i >= len(all_states)):
#             break
            
# for i in range(len(states_list[desired_method])):
#     args.append(states_list[desired_method][i])
    
# args = tuple(args)
# deneme = np.concatenate(args,axis=0)

# for i in range(1,len(Tf_list)):
#     Tf += Tf_list[i]
#     N += int(Tf_list[i]/dtau)
# t = linspace(0,Tf,num = N)

# nested = np.c_[t.reshape(-1,1), deneme[:,0:6].reshape(-1,6)]


# import pickle 
# with open('states.pickle', 'wb') as f:
#     pickle.dump(nested, f)
    
    
# with open('states.pickle', 'rb') as f:
#     states = pickle.load(f)

# # json_output = 'var sim_data = ' + json.dumps(nested.T.tolist(), indent=2)

# # # Writing to sample.json 
# # with open("visualizer/nonlinear_sim_log.js", "w") as outfile: 
# #     outfile.write(json_output) 

### Bobzwik Trajectory Algorithm
#### (Needs to be improved)

In [ ]:
# from waypoints import makeWaypoints

# flight_columns = ['x0', 'y0', 'z0', 'x_dot0','y_dot0','z_dot0', 'phi0','theta0','yaw0', 'phi_dot0','theta_dot0','yaw_dot0', 
#                  'xf', 'yf', 'zf', 'x_dotf','y_dotf','z_dotf','x_ddotf','y_ddotf','z_ddotf',
#                  'pos_diffx','pos_diffy','pos_diffz','time_rate','t', 'Tf', 
#                  'xp', 'yp', 'zp', 'x_dotp','y_dotp','z_dotp','x_ddotp','y_ddotp','z_ddotp', 'u_abs_p', 
#                  'r_std', 'phi_std', 'theta_std', 'psi_std', 'controller_ID']

# cost_columns = ["MAX", "MIX", "DICE", "DT", "FOREST", "Backstepping_1", "Backstepping_2", "Backstepping_3", "Backstepping_4"]
# flight_filename = "flight_gauss.csv"
# cost_filename = 'method_cost_list.csv'

# # STATUS = "DATA_COLLECTION"
# #STATUS = "SINGLE_SHOT"
# STATUS = "TEST"
# Controllers = ["Backstepping_1", "Backstepping_2", "Backstepping_3", "Backstepping_4"]
# METHOD_LIST = ["MAX", "MIX", "DICE"]#, "DT", "FOREST", "Backstepping_1", "Backstepping_2", "Backstepping_3", "Backstepping_4"]
# count_dict = {"Backstepping_1": 0, "Backstepping_2": 0, "Backstepping_3": 0, "Backstepping_4": 0}
# U_dict = {"Backstepping_1": 0, "Backstepping_2": 0, "Backstepping_3": 0, "Backstepping_4": 0}
# method_cost_list = {"MAX":[], "MIX":[], "DICE":[], "DT":[], "FOREST":[], "Backstepping_1":[], "Backstepping_2":[], "Backstepping_3":[], "Backstepping_4":[]}
# dtau = 1e-3
# Tf = 18

# if STATUS == "TEST":        
#     #Decision Tree classifier
#     decision_tree = joblib.load('Classifier/decision_tree.sav')
#     #To normalize data
#     scaler = joblib.load('Classifier/std_scaler.bin')

#     #Neural network
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     model = Net()
#     model = model.to(device)
#     model.load_state_dict(torch.load('Classifier/best_model.pt'))
#     model.eval()

    
# trajSelect = np.zeros(3)
# # Select Position Trajectory Type (0: hover,                    1: pos_waypoint_timed,      2: pos_waypoint_interp,    
# #                                  3: minimum velocity          4: minimum accel,           5: minimum jerk,           6: minimum snap
# #                                  7: minimum accel_stop        8: minimum jerk_stop        9: minimum snap_stop
# #                                 10: minimum jerk_full_stop   11: minimum snap_full_stop
# #                                 12: pos_waypoint_arrived
# trajSelect[0] = 6         
# # Select Yaw Trajectory Type      (0: none                      1: yaw_waypoint_timed,      2: yaw_waypoint_interp     3: follow          4: zero)
# trajSelect[1] = 4           
# # Select if waypoint time is used, or if average speed is used to calculate waypoint time   (0: waypoint time,   1: average speed)
# trajSelect[2] = 1  

# state0 = [2.,2.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
# newTraj = Trajectory(state0, trajSelect)

# U_list = []

# print ("RUN MODE: ", STATUS)

# N = int(Tf/dtau)
# t = linspace(0,Tf,num = N)

# # xd, yd, zd, psid = zeros(t.shape), zeros(t.shape), zeros(t.shape), zeros(t.shape)
# # xd_dot, yd_dot, zd_dot, psid_dot = zeros(t.shape), zeros(t.shape), zeros(t.shape), zeros(t.shape)
# # xd_ddot, yd_ddot, zd_ddot, psid_ddot = zeros(t.shape), zeros(t.shape), zeros(t.shape), zeros(t.shape)
# # xd_dddot, yd_dddot, zd_dddot = zeros(t.shape), zeros(t.shape), zeros(t.shape)
# # xd_ddddot, yd_ddddot, zd_ddddot = zeros(t.shape), zeros(t.shape), zeros(t.shape)

# i = 0
# ts = 0

# prev_traj = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# U = [1.,0.,0.,0.]

# t_targets, waypoints, yaw_targets, v_average = makeWaypoints()

# print ("")
# print ("-"*120)
# print ("Initial, x: {0:.3}, y: {1:.3}, z: {2:.3}, yaw: {3:.3} in {4:.3} s."
#            .format(waypoints[0][0], waypoints[0][1], waypoints[0][2], yaw_targets[0], t_targets[0]))
# for i in range(1, len(waypoints)):
#     print ("Waypoint, x: {0:.3}, y: {1:.3}, z: {2:.3}, yaw: {3:.3} in {4:.3} s."
#            .format(waypoints[i][0], waypoints[i][1], waypoints[i][2], yaw_targets[i], t_targets[i]))


# for method in METHOD_LIST:
#     print ("\nTEST MODE: ", method)
#     state = np.copy(state0)
#     costValue = 0.
        
#     for i in range(N):
#         t_current = t[i]
#         time_rate = float(t_current / Tf)
#         U0 = np.copy(U)
#         Upr_abs_sum = np.sum(np.abs(U0))
        
#         wind_direct = (np.random.randint(1,360)/180)*pi #Deg2Rad
#         wind_noisev = wind_shear(state[2],state[3],state[4],state[5],state[6],state[7],state[8],wind_direct)
        
        
#         pos_des, vel_des, acc_des, euler_des = newTraj.desiredState(t_current, dtau, state0)
#         xd, yd, zd = pos_des[0], pos_des[1], pos_des[2]
#         xd_dot, yd_dot, zd_dot = vel_des[0], vel_des[1], vel_des[2]
#         xd_ddot, yd_ddot, zd_ddot = acc_des[0], acc_des[1], acc_des[2]
#         xd_dddot, yd_dddot, xd_ddddot, yd_ddddot = 0, 0, 0, 0
#         psid, psid_dot, psid_ddot = euler_des[2], 0, 0
        
#         current_traj = [xd, yd, zd, xd_dot, yd_dot, zd_dot, xd_ddot, yd_ddot, zd_ddot,
#                      xd_dddot, yd_dddot, xd_ddddot, yd_ddddot,
#                      psid, psid_dot, psid_ddot]

#         state[6] += wind_noisev[0]
#         state[7] += wind_noisev[1]
#         state[8] += wind_noisev[2]
        
#         X_test = np.array([state[0], state[1], state[2], state[6], state[7], state[8], state[3], state[4], state[5], state[9], state[10], state[11],
#                     current_traj[0], current_traj[1], current_traj[2], current_traj[3], current_traj[4], current_traj[5], current_traj[6], current_traj[7], current_traj[8],
#                     state[0]-current_traj[0], state[1]-current_traj[1], state[2]-current_traj[2], time_rate, t_current, Tf, 
#                     prev_traj[0], prev_traj[1], prev_traj[2], prev_traj[3], prev_traj[4], prev_traj[5], prev_traj[6], prev_traj[7], prev_traj[8], 
#                     Upr_abs_sum, wind_direct]).reshape(1,-1)

#     #             X_test = scaler.transform(X_test)


#         if method == "MAX":
#             pred = predict(X_test, model, device, method)
#             cont = Controllers[pred]
#             U = get_control_input(cont, Controllers, U0, current_traj, state)
#             count_dict[cont] += 1

#         elif method == "DICE":
#             pred = predict(X_test, model, device, method)
#             cont = Controllers[pred]
#             U = get_control_input(cont, Controllers, U0, current_traj, state)
#             count_dict[cont] += 1

#         elif method == "MIX":
#             probs = predict(X_test, model, device, method)
#             U = 0
#             for k in range(4):
#                 cont = Controllers[k]
#                 cont_prob = probs[k]
#                 U_single = get_control_input(cont, Controllers, U0, current_traj, state)
#                 U = U + (cont_prob * U_single)

#         elif method == "DT":
#             pred = decision_tree.predict(X_test)[0]
#             cont = Controllers[pred]
#             U = get_control_input(cont, Controllers, U0, current_traj, state)
#             count_dict[cont] += 1

#         elif method == "FOREST":
#             pred = random_forest.predict(X_test)[0]
#             cont = Controllers[pred]
#             U = get_control_input(cont, Controllers, U0, current_traj, state)
#             count_dict[cont] += 1
#         else:
#             #This time a controller info comes in method variable
#             U = get_control_input(method, Controllers, U0, current_traj, state)

#         sol = integrate.solve_ivp(fun=model_dynamics, t_span=(0, 1e-6), y0=state, args = (U,))
#         state = sol.y[:,-1]


#         if (np.abs(state[3]) > pi/2)  | (np.abs(state[4]) > pi/2):
#             costValue = 1e6
#             method_cost_list[method].append(costValue)
#             break
#         else:
#             position_tracking_error = np.power((current_traj[0]-state[0]),2) + np.power((current_traj[1]-state[1]),2) + np.power((current_traj[2]-state[2]),2)
#             velocity_tracking_error = np.power((current_traj[3]-state[6]),2) + np.power((current_traj[4]-state[7]),2) + np.power((current_traj[5]-state[8]),2)
#             angular_error = state[3]**2 + state[4]**2 + state[5]**2
#             cont_input = U[0]**2 + U[1]**2 + U[2]**2 + U[3]**2
#             costValue = costValue + (position_tracking_error +  .25*angular_error)
            
            
            
#         prev_traj = np.copy(current_traj)
        
        
#     print ("Cost Value: ", costValue)
#     method_cost_list[method].append(costValue)
#     if (method == "MAX" or method == "DICE" or method == "DT" or method == "FOREST"):
#         print ("How many times Backstepping_1 is called?: ", count_dict["Backstepping_1"])
#         print ("How many times Backstepping_2 is called?: ", count_dict["Backstepping_2"])
#         print ("How many times Backstepping_3 is called?: ", count_dict["Backstepping_3"])
#         print ("How many times Backstepping_4 is called?: ", count_dict["Backstepping_4"])
#     print ("Final state, x: {0:.3}, y: {1:.3}, z: {2:.3}, phi: {3:.3}, theta: {4:.3}, psi: {5:.3}, vx: {6:.3}, vy: {7:.3}, vz: {8:.3}, p: {9:.3}, q: {10:.3}, r: {11:.3}".format(state[0],state[1],state[2],state[3],state[4],state[5], state[6],state[7],state[8], state[9],state[10],state[11]))
